In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision #contains
#definitions of models for addressing different tasks, including: image classification, pixelwise semantic segmentation,
from torch import nn,optim
from time import time
from torch.utils.data import TensorDataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
x=np.load('dataset.npy')
y=np.load('labels.npy')
print(type(x))
print(type(y))
#batch_size = 128
'''transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])'''
dataset = TensorDataset(torch.from_numpy(x[0:50000]).unsqueeze(1), torch.from_numpy(y[0:50000]))
trainloader=torch.utils.data.DataLoader(dataset,batch_size=1000,shuffle=True)
#valloader=torch.utils.data.DataLoader(y,batch_size=64,shuffle=True)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [3]:
img1=read_image("im-1.png")
img2=read_image("im-2.png")
img3=read_image("im-3.png")
img4=read_image("im-4.png")
img5=read_image("im-5.png")
#valset=torch.cat([img1,img2,img3,img4,img5],0)
valset=TensorDataset(torch.from_numpy(x[50000:59994]).unsqueeze(1), torch.from_numpy(y[50000:59994]))
valloader=torch.utils.data.DataLoader(valset,batch_size=1000,shuffle=True)

In [4]:
dataiter=iter(trainloader)
images,labels=dataiter.next()
print(images.shape)
print(labels.shape)

torch.Size([1000, 1, 28, 28])
torch.Size([1000])


In [6]:
input_size=784      #structure of the neural network
hidden_sizes=[128,64]
output_size=10
model=nn.Sequential(nn.Linear(input_size,hidden_sizes[0]),
nn.ReLU(),
nn.Linear(hidden_sizes[0],hidden_sizes[1]),
nn.ReLU(),
nn.Linear(hidden_sizes[1],output_size),
nn.LogSoftmax(dim=1))#softmax function takes a vector/tensor and returns a probability distribution of n possible outcomes
print(model)


Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)


In [7]:
criterion = nn.NLLLoss()#negative log likelihood
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logps = model(images.float())
loss = criterion(logps,labels)

In [8]:
print('Before backward pass: \n', model[0].weight.grad)
loss.backward()
print('After backward pass: \n', model[0].weight.grad)

Before backward pass: 
 None
After backward pass: 
 tensor([[-3.0481e-02,  0.0000e+00,  0.0000e+00,  ..., -1.0865e+00,
          1.3389e-04, -4.6979e-01],
        [ 6.1478e-02,  0.0000e+00,  0.0000e+00,  ...,  1.3885e+00,
          1.9286e-05,  6.0884e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  7.3089e-01,
          0.0000e+00,  1.8236e-01],
        ...,
        [ 3.9683e-02,  0.0000e+00,  0.0000e+00,  ...,  1.4644e+00,
          5.3031e-05,  4.1940e-01],
        [ 2.5094e-02,  0.0000e+00,  0.0000e+00,  ...,  3.0216e-01,
          0.0000e+00,  1.8605e-01],
        [ 6.3802e-03,  0.0000e+00,  0.0000e+00,  ...,  1.2840e+00,
         -1.1139e-04,  5.4375e-01]])


In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.003,momentum=0.9)#momentum
#is a variant of stochastic gradient descent It replaces the 
# gradient with a momentum which is an aggregate of gradients 
# optimizers are algorithms that adjust the model’s parameters during training to minimize a loss function
time0 = time()
epochs = 100
losses=[]
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
    
        optimizer.zero_grad()
        
        output = model(images.float())
        loss = criterion(output, labels)
        #losses.append(loss.detach().numpy())
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        losses.append(running_loss/len(trainloader))

    else:
        if e%10==0:
            print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
        
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 2.2722161424160006
Epoch 10 - Training loss: 0.12314142733812332
Epoch 20 - Training loss: 0.080153953358531
Epoch 30 - Training loss: 0.0541091026365757
Epoch 40 - Training loss: 0.03908211566507816
Epoch 50 - Training loss: 0.026891570203006268
Epoch 60 - Training loss: 0.020397195760160684
Epoch 70 - Training loss: 0.013773904172703624
Epoch 80 - Training loss: 0.009223891431465745
Epoch 90 - Training loss: 0.00626274255104363

Training Time (in minutes) = 4.5756852388381954


In [ ]:
plt.plot(range(epochs),losses)
plt.ylabel("loss/error")
plt.xlabel('Epoch')

In [10]:
correct_count, all_count = 0, 0
for images,labels in valloader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img.float())

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count)*100,"%")

Number Of Images Tested = 9994

Model Accuracy = 96.26776065639385 %


In [ ]:
torch.save(model,'number_recognition_model.pt')

In [ ]:
img1=(read_image("im-1.png"))
img2=read_image("im-2.png")
img3=read_image("im-3.png")
img4=read_image("im-4.png")
img5=read_image("im-5.png")
#s=np.load("dataset.npy")
#s.shape
valset2=torch.cat([img1,img2,img3,img4,img5],0)
#valset2.shape

'''for images in valset2:
     img=images[i].view(1,28)
     with torch.no_grad():
            logps=model(img.float())
        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        print(pred_label)'''
        
            
        
#print(model_loaded(valset2))
